In [10]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a teacher"},
    {"role": "user", "content": "What is the capital of france?"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [5]:
test = "SELECT [column1], [aggregation]([column2]) FROM {table} WHERE [condition] GROUP BY [column1]"
formatted_query = test.format(table="abc")
print(formatted_query)


SELECT [column1], [aggregation]([column2]) FROM abc WHERE [condition] GROUP BY [column1]


### Creating Database from CSV files

In [13]:
import sqlite3
import os
import pandas as pd

def csv_folder_to_database_custom_schema(folder_path, db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            table_name = os.path.splitext(filename)[0].replace(" ", "_").replace("-", "_")
            print(table_name)

            cursor.execute(f"""
                    CREATE TABLE IF NOT EXISTS {table_name}(
                        day INTEGER NOT NULL CHECK (day BETWEEN 1 AND 31),
                        month TEXT NOT NULL,
                        year INTEGER NOT NULL,
                        dayname TEXT NOT NULL,
                        season INTEGER NOT NULL,
                        stadium TEXT NOT NULL,
                        city TEXT NOT NULL,
                        state TEXT NOT NULL,
                        attendance INTEGER NOT NULL,
                        capacity INTEGER NOT NULL,
                        summary TEXT
                    )
                """)
            print(f"Custom schema applied for table '{table_name}'.")

            if 'game_id' in df.columns:
                df = df.drop(columns=['game_id'])

            for _, row in df.iterrows():
                placeholders = ', '.join(['?'] * len(row))
                column_names = ', '.join(row.index)
                insert_query = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"
                cursor.execute(insert_query, tuple(row))

    conn.commit()
    print("All CSV files have been successfully imported with the custom schema.")
    return conn

folder_path = r'D:\LLMTables\LLMTablesQA\Question Generation\TestTables_5'
db_path = r'D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_3.db'

conn = csv_folder_to_database_custom_schema(folder_path, db_path)



sportset_2
Custom schema applied for table 'sportset_2'.
sportset_coldtemp_30_13
Custom schema applied for table 'sportset_coldtemp_30_13'.
sportset_midwest_30_8
Custom schema applied for table 'sportset_midwest_30_8'.
sportset_northeast_30_1
Custom schema applied for table 'sportset_northeast_30_1'.
sportset_west_30_4
Custom schema applied for table 'sportset_west_30_4'.
All CSV files have been successfully imported with the custom schema.


In [12]:
import sqlite3

db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_3.db" 
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]
print("Tables in the database:", tables)


conn.close()


Tables in the database: ['sportset_2', 'sportset_coldtemp_30_13', 'sportset_midwest_30_8', 'sportset_northeast_30_1', 'sportset_west_30_4']


In [ ]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
result = pd.read_sql_query("SELECT dayname FROM sportset_2 WHERE attendance > 17000 AND state = 'Tennessee'", conn)
str(result)

'    dayname\n0  Saturday\n1  Saturday\n2   Tuesday'

In [13]:
import pandas as pd

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def get_table_schema_and_rows(table_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    schema = cursor.fetchall()
    schema_dict = {col[1]: col[2] for col in schema} 
    query = f"SELECT * FROM {table_name} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    return schema_dict, df

table_name = tables[0]
schema, rows = get_table_schema_and_rows(table_name)
print("Schema:", schema)
print("\nSample Rows:")
print(rows)
conn.close()


Schema: {'day': 'INTEGER', 'month': 'TEXT', 'year': 'INTEGER', 'dayname': 'TEXT', 'season': 'INTEGER', 'stadium': 'TEXT', 'city': 'TEXT', 'state': 'TEXT', 'attendance': 'INTEGER', 'capacity': 'INTEGER', 'summary': 'TEXT'}

Sample Rows:
   day     month  year    dayname  season                  stadium  \
0    5  February  2016     Friday    2015             Amway Center   
1    9  November  2018     Friday    2018  American Airlines Arena   
2    4  December  2016     Sunday    2016  Chesapeake Energy Arena   
3   28   October  2017   Saturday    2017               FedExForum   
4    6     March  2019  Wednesday    2018     Smoothie King Center   

            city      state  attendance  capacity  \
0        Orlando    Florida       16600     18800   
1          Miami    Florida       19600     19600   
2  Oklahoma City   Oklahoma       18200     18200   
3        Memphis  Tennessee       17000     17800   
4    New Orleans  Louisiana       14700     16900   

                        

In [9]:
conn.close()

### Generating queries

In [ ]:
from openai import OpenAI
import random
import json
import sqlite3
db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
client = OpenAI()

def generate_queries(table_name, schema, rows):
    row_samples = [rows.sample(1, random_state=random.randint(1, 1000)).to_dict(orient="records")[0] for _ in range(5)]
    print(f"Generating queries for {table_name}")
    messages = [
        {"role": "system", "content": "Act as an expert in SQL and databases. Please give valid output JSON."},
        {"role": "user", "content": f"""
         Read the table schema and 5 rows given from the table carefully and understand it correctly - 
         
         Table Schema:
         {json.dumps(schema, indent=4)}

         ROW DATA:
         {json.dumps(row_samples, indent=4)}

         SQL TEMPLATE:

         SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] LIKE '%pattern%' GROUP BY [column1]

         Instruction:
            Please use the information from the table and data provided to fill in the placeholders in the template. Each SQL query should return useful, non-null results and should be a logical grouping based on the table data.
            The query should return a single result using: 
            - A specific column (e.g., city, attendance, capacity, etc.)
            - An aggregate function (e.g., COUNT(), SUM(), MAX(), etc.)
            - ensure that it is an executable SQL Queries with a non null result
            - FOLLOW THE TEMPLATE DILIGENTLY
            - Ensure that the table name in the query is correct

            Follow the template given and try to fill in the placeholders in a way that can lead to logical and complex queries.
            Ensure that the queries generate deterministic answers, such as a single count, maximum, or specific column value (e.g., "How many games were held in Orlando?").
            Generate 5 such SQL queries.

            Example Queries- 
            "SELECT state, SUM(attendance) FROM sportset_2 WHERE stadium LIKE '%Center%' GROUP BY state HAVING SUM(attendance) > 30000",
            "SELECT COUNT(*) FROM [table] WHERE summary LIKE '%[team name]%' AND month = 'February'",
            "SELECT * FROM [table] WHERE summary LIKE '%[team name]%'"
            "SELECT city, COUNT(*) FROM [table] WHERE summary LIKE '%injured%' GROUP BY city"


            Please follow the Response format while answering :
            Query: <Single Liner SQL Query>
            Table: {table_name}
         """}
    ]

    chat_completion, *_ = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        response_format={"type": "json_object"}

    ).choices
    content = chat_completion.message.content
    reply = json.loads(content)
    return reply

queries_json = generate_queries("sportset_midwest_30_8", schema, rows)

print("Generated JSON Output:")
print(json.dumps(queries_json, indent=4))

Generating queries for sportset_midwest_30_8
Generated JSON Output:
{
    "queries": [
        {
            "Query": "SELECT state, SUM(attendance) FROM sportset_midwest_30_8 WHERE stadium LIKE '%Energy Arena%' GROUP BY state",
            "Table": "sportset_midwest_30_8"
        },
        {
            "Query": "SELECT city, COUNT(*) FROM sportset_midwest_30_8 WHERE summary LIKE '%Grizzlies%' GROUP BY city HAVING COUNT(*) > 1",
            "Table": "sportset_midwest_30_8"
        },
        {
            "Query": "SELECT month, MAX(attendance) FROM sportset_midwest_30_8 WHERE city LIKE '%Memphis%' GROUP BY month",
            "Table": "sportset_midwest_30_8"
        },
        {
            "Query": "SELECT year, AVG(capacity) FROM sportset_midwest_30_8 WHERE dayname LIKE '%Saturday%' GROUP BY year",
            "Table": "sportset_midwest_30_8"
        },
        {
            "Query": "SELECT stadium, COUNT(*) FROM sportset_midwest_30_8 WHERE summary LIKE '%triple-double%' GROUP BY

In [23]:
db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT dayname, COUNT(*) FROM sport_events WHERE month = 'November' AND year = 2018 GROUP BY dayname").fetchall()

OperationalError: no such table: sport_events

In [63]:
basic_selection = [
    "SELECT [column] FROM [table] WHERE [conditions]",
    "SELECT [column] FROM [table] WHERE [conditions] ORDER BY [column]",
    "SELECT [column] FROM [table] WHERE [conditions] ORDER BY [ ] LIMIT [ ]"
]
and_queries = [
    "SELECT [column] FROM [table] WHERE [condition1] AND [condition2]",
    "SELECT [column] FROM [table] WHERE [condition1] AND [condition2] ORDER BY [column] LIMIT [n]"
]

or_queries = [
    "SELECT [column] FROM [table] WHERE [condition1] OR [condition2]",
    "SELECT [column] FROM [table] WHERE [condition1] OR [condition2] ORDER BY [column] LIMIT [n]",
    "SELECT [column] FROM [table] WHERE [condition1] OR [condition2] ORDER BY [column]"
]

advanced_and_or_queries = [
    "SELECT [column] FROM [table_name] WHERE [condition1 AND condition2 AND condition3]",
    "SELECT [column] FROM [table_name] WHERE [condition1] OR [condition2] OR [condition3]",
    "SELECT [column] FROM [table_name] WHERE [condition1] OR [condition2] OR [condition3] ORDER BY [column]",
    "SELECT [column] FROM [table_name] WHERE [condition1 AND condition2] OR [condition3]",
    "SELECT [column] FROM [table_name] WHERE [condition1 AND condition2] OR [condition3 AND column4]",
    "SELECT [column] FROM [table_name] WHERE [condition1 OR condition2] AND [condition3 OR column4]",
]

distinct_queries = [
    "SELECT DISTINCT [column] FROM [table] WHERE [conditions]",
    "SELECT DISTINCT [column] FROM [table] WHERE [conditions] ORDER BY [ ]",
    "SELECT DISTINCT [column] FROM [table] WHERE [conditions] ORDER BY [ ] LIMIT [n]"
]

aggregation_queries = [
    "SELECT MAX/MIN/AVG/SUM/COUNT([column]) FROM [table] WHERE [conditions]",
    "SELECT MAX/MIN/AVG/SUM/COUNT([column]) FROM [table] WHERE [conditions] LIMIT [ ]",
    "SELECT MAX/MIN/AVG/SUM/COUNT([column]) FROM [table] WHERE [conditions] ORDER BY [ ]"
]

case_queries = [
    "SELECT [column], CASE WHEN [condition1] THEN [value1] WHEN [condition2] THEN [value2] ELSE [default value] END AS [new column] FROM [table] WHERE [conditions]",
]

like_queries = [
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%'",
    "SELECT [column] FROM [table] WHERE [column] NOT LIKE '%pattern%'",
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern1%' OR [column] LIKE '%pattern2%'",
    "SELECT [column] FROM [table] WHERE [column] NOT LIKE '%pattern1%' AND [column] LIKE '%pattern2%'",
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%' AND [another_column] = [value]",
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%' OR [another_column] BETWEEN [value1] AND [value2]",
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%' ORDER BY [column]",
    "SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%' ORDER BY [column] LIMIT [n]",
]

group_by_queries = [
    # Basic Group By Queries
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [condition] GROUP BY [column1]",
    "SELECT [column1], COUNT(*) FROM [table] WHERE [condition] GROUP BY [column1]",
    "SELECT DISTINCT [column1], [aggregation]([column2]) FROM [table] WHERE [conditions] GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [condition] GROUP BY [column1] ORDER BY [aggregation]([column2]) LIMIT [n]"
    
    # Group By with AND Conditions
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [condition1] AND [condition2] GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [condition1] AND [condition2] GROUP BY [column1] HAVING COUNT([column2]) > [value]",
    
    # Group By with OR Conditions
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [condition1] OR [condition2] GROUP BY [column1]",
    "SELECT [column1], COUNT(*) FROM [table] WHERE [condition1] OR [condition2] GROUP BY [column1]",
    
    # Group By with LIKE Conditions
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] LIKE '%pattern%' GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] NOT LIKE '%pattern%' GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] LIKE '%pattern%' AND [column2] BETWEEN [value1] AND [value2] GROUP BY [column1]",
    "SELECT [column1], COUNT(*) FROM [table] WHERE [column1] LIKE '%pattern%' GROUP BY [column1] HAVING COUNT(*) > [value]",
    "SELECT [column1], [aggregation1]([column2]) AS [alias1], [aggregation2]([column3]) AS [alias2] FROM [table] GROUP BY [column1] HAVING [aggregation1]([column2]) > [value]",
    "SELECT [column1], [aggregation]([column2]) FROM [table1] WHERE [column1] IN (SELECT [column3] FROM [table2] WHERE [condition]) GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table1] WHERE [column1] IN (SELECT [column3] FROM [table2] WHERE [condition]) GROUP BY [column1] ORDER BY [aggregation]([column2]) DESC LIMIT [n]",

    # Group By with CASE Statements
    "SELECT [column1], CASE WHEN [condition1] THEN [value1] WHEN [condition2] THEN [value2] ELSE [default_value] END AS [new_column] FROM [table] GROUP BY [column1]",
    "SELECT [column1], COUNT(CASE WHEN [condition1] THEN 1 ELSE NULL END) AS count_condition1 FROM [table] GROUP BY [column1]",
    
    # Group By with Subqueries
    "SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] IN (SELECT [sub_column] FROM [sub_table] WHERE [sub_condition]) GROUP BY [column1]",
    "SELECT [column1], COUNT(*) FROM [table] WHERE EXISTS (SELECT 1 FROM [sub_table] WHERE [table].[column] = [sub_table].[column] AND [sub_condition]) GROUP BY [column1]",

    "SELECT [column1], [aggregation]([column2]) FROM [table_name] WHERE [condition1 AND condition2 AND condition3] GROUP BY [column1]",
    "SELECT [column1], [aggregation]([column2]) FROM [table_name] WHERE [condition1 AND condition2 AND condition3] GROUP BY [column1] ORDER BY [column1] LIMIT [n]"
]

query_templates = [basic_selection, and_queries, or_queries, advanced_and_or_queries, distinct_queries, aggregation_queries, case_queries, like_queries, group_by_queries]

In [68]:
import random
selected_template = random.choice(random.choice(query_templates))
selected_template


'SELECT [column] FROM [table] WHERE [condition1] OR [condition2] ORDER BY [column] LIMIT [n]'

In [72]:
from openai import OpenAI
import random
import json
import sqlite3
db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
client = OpenAI()

def generate_queries(table_name, schema, rows, query_templates):
    row_samples = [rows.sample(1, random_state=random.randint(1, 1000)).to_dict(orient="records")[0] for _ in range(5)]
    print(f"Generating queries for {table_name}")
    selected_template = random.choice(random.choice(query_templates))
    print(selected_template)
    messages = [
        {"role": "system", "content": "Act as an expert in SQL and databases. Please give valid output JSON."},
        {"role": "user", "content": f"""
         Read the table schema and 5 rows given from the table carefully and understand it correctly - 
         
         Table Schema:
         {json.dumps(schema, indent=4)}

         ROW DATA:
         {json.dumps(row_samples, indent=4)}

         SQL TEMPLATE:
         {selected_template}
         
         Instruction:
         Your task is to fill in the placeholders in the query templates and create meaningful & logical SQL queries.
            1. Please use the information from the table and data provided to fill in the placeholders in the template. Each SQL query should only return a single result using either:
            - A specific column (e.g., city, attendance, capacity, etc.)
            - An aggregate function (e.g., COUNT(), SUM(), MAX(), etc.)

            2. Follow the template given and try to fill in the placeholders in a way that can lead to logical and complex queries.
            3. Add HAVING, LIKE and DISTINCT to the template to generate queries as you wish. Note that HAVING should be followed by a GROUP BY always.
            4. Make use of AND,OR and NOT at will in the conditions.
            5. If you're adding LIMIT make sure it follows a meaningful ORDER BY so that the answer is not dependent on internal ordering of the database.
            6. Try to make the query set as diverse as possible while not deviating much from the template given to you.
            7. Ensure that the queries generate deterministic answers, such as a single count, maximum, or specific column value (e.g., "How many games were held in Orlando?").
            8. Ensure all queries are syntactically correct
            
            PLEASE FOLLOW THE TEMPLATE!

            Please follow the Response format while answering :
            Query: <Single Liner SQL Query>
            Table: {table_name}

            Generate 2 such SQL Queries.
         """}
    ]

    chat_completion, *_ = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        response_format={"type": "json_object"}

    ).choices
    content = chat_completion.message.content
    reply = json.loads(content)
    return reply

queries_json = generate_queries("sportset_midwest_30_8", schema, rows, query_templates)

print("Generated JSON Output:")
print(json.dumps(queries_json, indent=4))

Generating queries for sportset_midwest_30_8
SELECT [column], CASE WHEN [condition1] THEN [value1] WHEN [condition2] THEN [value2] ELSE [default value] END AS [new column] FROM [table] WHERE [conditions]
Generated JSON Output:
{
    "queries": [
        {
            "Query": "SELECT city, COUNT(*) AS total_games FROM sportset_midwest_30_8 GROUP BY city HAVING total_games > 1",
            "Table": "sportset_midwest_30_8"
        },
        {
            "Query": "SELECT summary, CASE WHEN attendance < capacity THEN 'Under Capacity' ELSE 'At Capacity' END AS attendance_status FROM sportset_midwest_30_8 WHERE season = 2016 AND city LIKE '%Oklahoma%' ORDER BY attendance DESC LIMIT 1",
            "Table": "sportset_midwest_30_8"
        }
    ]
}


In [74]:
import sqlite3
db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT city, COUNT(*) AS total_games FROM sportset_midwest_30_8 GROUP BY city HAVING total_games > 1").fetchall()

[('Chicago', 3),
 ('Cleveland', 3),
 ('Detroit', 4),
 ('Indianapolis', 12),
 ('Milwaukee', 2),
 ('Minneapolis', 6)]

In [58]:
from openai import OpenAI
import random
import json
import sqlite3
db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
client = OpenAI()

def generate_queries(table_name, schema, rows):
    row_samples = [rows.sample(1, random_state=random.randint(1, 1000)).to_dict(orient="records")[0] for _ in range(5)]
    print(f"Generating queries for {table_name}")
    messages = [
        {"role": "system", "content": "Act as an expert in SQL and databases. Please give valid output JSON."},
        {"role": "user", "content": f"""
        Read the table schema and 5 rows given from the table carefully and understand it correctly. Pay special attention to identifying key patterns in columns like `summary` and `stadium` for LIKE queries.

            ### Table Schema:
            {json.dumps(schema, indent=4)}

            ### ROW DATA:
            {json.dumps(row_samples, indent=4)}

            ### SQL TEMPLATE:

            SELECT [column1], [aggregation]([column2]) FROM [table] WHERE [column1] LIKE '%pattern%' GROUP BY [column1]

            ### Instruction:
            Your task is to fill in the placeholders in the query templates and create meaningful & logical SQL queries. Focus on making LIKE queries insightful by identifying specific patterns from the schema and sample data. 

            #### Key Instructions for LIKE Queries:
            1. Analyze the sample data and schema to identify recurring patterns or terms. The following are **high-priority patterns**:
            - Team names from the `summary` column (e.g., "Boston Celtics", "Los Angeles Lakers", etc.).
            - Common terms in the `stadium` column, such as "Centre" or "Arena".
            - IT IS MOST IMPORTANT TO FOCUS ON CREATING QUERIES WITH TEAM NAMES

            2. Always generate queries that:
            - Return meaningful results from the table (e.g., using conditions like WHERE and HAVING with aggregations).
            - Use diverse LIKE patterns based on real data patterns (team names, keywords, locations, etc.).
            - Include conditions such as AND, OR, and NOT for more logical complexity.

            3. Add additional clauses like HAVING, DISTINCT, and LIMIT to make the queries more diverse and realistic.

            4. Ensure all queries adhere to the template provided:
            - Use a single result column or aggregation function.
            - Use deterministic conditions (e.g., COUNT, SUM, MAX, etc.).
            - Ensure a meaningful ORDER BY clause for any LIMIT statements.

            #### Example Queries:
            - "SELECT state, SUM(attendance) FROM sportset_2 WHERE stadium LIKE '%Center%' GROUP BY state HAVING SUM(attendance) > 30000"
            - "SELECT COUNT(*) FROM [table] WHERE summary LIKE '%Boston Celtics%' AND month = 'February'"
            - "SELECT stadium, MAX(capacity) FROM [table] WHERE stadium LIKE '%Arena%'"
            - "SELECT * FROM [table] WHERE summary LIKE '%Los Angeles Lakers%'"

            ### Response Format:
            Query: <Single Liner SQL Query>
            Table: {table_name}

            ### Task:
            Generate 5 such SQL Queries.

         """}
    ]

    chat_completion, *_ = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        response_format={"type": "json_object"}

    ).choices
    content = chat_completion.message.content
    reply = json.loads(content)
    return reply

queries_json = generate_queries("sportset_2", schema, rows)

print("Generated JSON Output:")
print(json.dumps(queries_json, indent=4))

Generating queries for sportset_2
Generated JSON Output:
{
    "queries": [
        {
            "Query": "SELECT city, SUM(attendance) FROM sportset_2 WHERE summary LIKE '%Los Angeles Clippers%' GROUP BY city HAVING SUM(attendance) > 10000",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT stadium, COUNT(*) FROM sportset_2 WHERE summary LIKE '%New Orleans Pelicans%' GROUP BY stadium HAVING COUNT(*) > 1",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT state, AVG(attendance) FROM sportset_2 WHERE stadium LIKE '%Center%' GROUP BY state ORDER BY AVG(attendance) DESC LIMIT 3",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT dayname, COUNT(*) FROM sportset_2 WHERE summary LIKE '%Utah Jazz%' AND month = 'March' GROUP BY dayname",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT stadium, MAX(capacity) FROM sportset_2 WHERE summary LIKE '%Houston Rocke

### Scaled Query Generation

In [59]:
def process_all_tables_and_save_simple(db_path, output_file):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    final_result = []

    for table_name in tables:
        print(f"Processing table: {table_name}")

        schema, rows = get_table_schema_and_rows(table_name)

        queries_json = generate_queries(table_name, schema, rows)

        for query in queries_json["queries"]:
            final_result.append({
                "table_name": query["Table"],
                "query": query["Query"] 
            })

    conn.close()

    with open(output_file, "w") as f:
        json.dump(final_result, f, indent=4)
    
    print(f"JSON output saved to {output_file}")

db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
output_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_simple_queries_output.json"
all_tables_result = process_all_tables_and_save_simple(db_path, output_file)

print("Generated Simplified JSON Output for All Tables:")
print(json.dumps(all_tables_result, indent=4))


Processing table: sportset_2
Generating queries for sportset_2
Processing table: sportset_coldtemp_30_13
Generating queries for sportset_coldtemp_30_13
Processing table: sportset_midwest_30_8
Generating queries for sportset_midwest_30_8
Processing table: sportset_northeast_30_1
Generating queries for sportset_northeast_30_1
Processing table: sportset_west_30_4
Generating queries for sportset_west_30_4
JSON output saved to D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_simple_queries_output.json
Generated Simplified JSON Output for All Tables:
null


In [82]:
def process_all_tables_and_save_simple(db_path, output_file):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    final_result = []

    for table_name in tables:
        print(f"Processing table: {table_name}")
        for i in range(5):
            print(f"  Generating queries for table '{table_name}', iteration {i + 1}")
            schema, rows = get_table_schema_and_rows(table_name)
            queries_json = generate_queries(table_name, schema, rows, query_templates)
            print(json.dumps(queries_json, indent=4))
            for query in queries_json["queries"]:
                normalized_query = {k.lower(): v for k, v in query.items()}
                final_result.append({
                    "table_name": normalized_query.get("table", table_name), 
                    "query": normalized_query.get("query", "Unknown Query")  
                })

    conn.close()
    with open(output_file, "w") as f:
        json.dump(final_result, f, indent=4)
    
    print(f"JSON output saved to {output_file}")

db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"  
output_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\multiple_query_type_simple_queries_output.json"
all_tables_result = process_all_tables_and_save_simple(db_path, output_file)

print("Generated Simplified JSON Output for All Tables:")
print(json.dumps(all_tables_result, indent=4))


Processing table: sportset_2
  Generating queries for table 'sportset_2', iteration 1
Generating queries for sportset_2
SELECT DISTINCT [column] FROM [table] WHERE [conditions] ORDER BY [ ] LIMIT [n]
{
    "queries": [
        {
            "Query": "SELECT DISTINCT city FROM sportset_2 WHERE state = 'Tennessee' ORDER BY city LIMIT 1",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT COUNT(*) FROM sportset_2 WHERE season = 2018 AND attendance > 19000 GROUP BY stadium HAVING COUNT(*) > 1",
            "Table": "sportset_2"
        }
    ]
}
  Generating queries for table 'sportset_2', iteration 2
Generating queries for sportset_2
SELECT [column] FROM [table] WHERE [column] LIKE '%pattern%' ORDER BY [column] LIMIT [n]
{
    "queries": [
        {
            "Query": "SELECT city FROM sportset_2 WHERE city LIKE '%Orlando%' ORDER BY attendance DESC LIMIT 1",
            "Table": "sportset_2"
        },
        {
            "Query": "SELECT MAX(attendanc

In [78]:
queries_json ={
    "queries": [
        {
            "query": "SELECT city FROM sportset_coldtemp_30_13 WHERE state = 'Illinois' AND attendance > 20000 ORDER BY attendance DESC LIMIT 1",
            "table": "sportset_coldtemp_30_13"
        },
        {
            "query": "SELECT COUNT(game_id) FROM sportset_coldtemp_30_13 WHERE month LIKE 'February' AND season = 2015 GROUP BY city HAVING COUNT(game_id) > 1",
            "table": "sportset_coldtemp_30_13"
        }
    ]
}

In [80]:
final_result = []
for query in queries_json["queries"]:
    # Normalize keys to handle case-insensitivity
    normalized_query = {k.lower(): v for k, v in query.items()}

    final_result.append({
                    "table_name": normalized_query.get("table", table_name),  # Default to current table name
                    "query": normalized_query.get("query", "Unknown Query")  # Default placeholder
    })


In [81]:
final_result

[{'table_name': 'sportset_coldtemp_30_13',
  'query': "SELECT city FROM sportset_coldtemp_30_13 WHERE state = 'Illinois' AND attendance > 20000 ORDER BY attendance DESC LIMIT 1"},
 {'table_name': 'sportset_coldtemp_30_13',
  'query': "SELECT COUNT(game_id) FROM sportset_coldtemp_30_13 WHERE month LIKE 'February' AND season = 2015 GROUP BY city HAVING COUNT(game_id) > 1"}]

### Execute Queries and Modify JSON

In [60]:
import sqlite3
import json

def execute_queries_and_update_json(db_path, input_json_file, output_json_file):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    with open(input_json_file, "r") as f:
        queries_json = json.load(f)

    updated_queries = []

    for entry in queries_json:
        table_name = entry["table_name"]
        query = entry["query"]
        print(query)
        
        try:
            cursor.execute(query)
            result = cursor.fetchall()
            print(result)
            if len(result) == 1 and len(result[0]) == 1:
                result_value = result[0][0]
            else:
                result_value = str(result)
        except Exception as e:
            result_value = f"Error: {str(e)}"

        updated_queries.append({
            "table_name": table_name,
            "query": query,
            "result": result_value
        })

    conn.close()

    with open(output_json_file, "w") as f:
        json.dump(updated_queries, f, indent=4)
    
    print(f"Updated JSON with results saved to {output_json_file}")
    return updated_queries

db_path = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\new_database_2.db"
input_json_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_simple_queries_output.json"  
output_json_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_queries_with_results.json" 

updated_result = execute_queries_and_update_json(db_path, input_json_file, output_json_file)

print("Updated JSON with Results:")
print(json.dumps(updated_result, indent=4))


SELECT state, SUM(attendance) FROM sportset_2 WHERE summary LIKE '%Los Angeles Clippers%' GROUP BY state HAVING SUM(attendance) > 15000
[('Florida', 16600), ('Louisiana', 16400), ('North Carolina', 17600)]
SELECT stadium, COUNT(game_id) FROM sportset_2 WHERE summary LIKE '%Miami Heat%' GROUP BY stadium ORDER BY COUNT(game_id) DESC LIMIT 3
[('Time Warner Cable Arena', 1), ('Spectrum Center', 1), ('Chesapeake Energy Arena', 1)]
SELECT city, AVG(attendance) FROM sportset_2 WHERE summary LIKE '%Oklahoma City Thunder%' GROUP BY city HAVING AVG(attendance) < 20000
[('Oklahoma City', 10920.0)]
SELECT month, SUM(capacity) FROM sportset_2 WHERE stadium LIKE '%Center%' GROUP BY month HAVING SUM(capacity) > 50000
[('December', 54200), ('March', 53800)]
SELECT dayname, COUNT(DISTINCT game_id) FROM sportset_2 WHERE summary LIKE '%Anthony Davis%' AND stadium LIKE '%Arena%' GROUP BY dayname ORDER BY COUNT(DISTINCT game_id) DESC
[('Sunday', 1)]
SELECT state, SUM(attendance) FROM sportset_coldtemp_30_1

### Convert SQL Query to Natural Language

In [61]:
from openai import OpenAI
import json

client = OpenAI()
def convert_sql_to_natural_language(input_json_file, output_json_file):
    with open(input_json_file, "r") as f:
        queries_json = json.load(f)
    
    updated_json = []
    
    for entry in queries_json:
        sql_query = entry["query"]
        table_name = entry["table_name"]
        result = entry.get("result", None)
        
        prompt = f"""
        You are an expert data scientist skilled in SQL and natural language processing. Your task is to convert SQL queries into natural language questions. 

        Here is the SQL Query: 
        {sql_query}
        The questions should:
        - Clearly represent the intent of the SQL query.
        - Translate technical terms, column headers, and values into natural, descriptive forms.
        - Avoid technical jargon unless absolutely necessary.
        - Ensure the question retains the same scope and meaning as the SQL query to avoid altering the query's answer.

        For example:
        SQL: SELECT COUNT(game_id) FROM sportset_2 WHERE city = 'Orlando' AND year = 2015
        Output: How many games took place in Orlando in the year 2015?

        Please convert this SQL query into a single natural language question. Ensure the column headers and values are human-readable. 
        Respond STRICTLY in the following format:
        Question: <Natural Language Question>
        """
        
        messages = [
        {"role": "system", "content": "Act as an expert data scientist skilled in SQL and natural language processing."},
        {"role": "user", "content": prompt}
        ]
        
        try:  
            chat_completion, *_ = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages
                ).choices
            content = chat_completion.message.content
            print(content)      
            question = content.replace("Question: ", "").strip()
        except Exception as e:
            question = f"Error generating question: {str(e)}"
        updated_json.append({
            "table_name": table_name,
            "query": sql_query,
            "result": result,
            "question": question
        })
    with open(output_json_file, "w") as f:
        json.dump(updated_json, f, indent=4)
    
    print(f"Updated JSON with natural language questions saved to {output_json_file}")
    return updated_json
input_json_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_queries_with_results.json"  
output_json_file = r"D:\LLMTables\LLMTablesQA\Question Generation\SQL\like_natural_language_output.json" 

updated_json = convert_sql_to_natural_language(input_json_file, output_json_file)

print("Updated JSON with Natural Language Questions:")
print(json.dumps(updated_json, indent=4))


Question: Which states had a total attendance of more than 15,000 for games involving the Los Angeles Clippers?
Question: What are the three stadiums that hosted the most games featuring the Miami Heat?
Question: In which cities did the average attendance for games featuring the Oklahoma City Thunder fall below 20,000?
Question: In which months did stadiums with "Center" in their name have a total capacity greater than 50,000?
Question: On which days did Anthony Davis play games at an arena, and how many distinct games were played on those days?
Question: Which states had a total attendance of more than 10,000 for games involving the Washington Wizards?
Question: Which stadium that includes 'Arena' in its name has hosted the most events?
Question: In which months did the Charlotte Hornets have an average attendance greater than 15,000?
Question: Which cities have a summary mentioning either the Jazz or the Hornets?
Question: On which days in April did stadiums with "Center" in their na

### Evaluation

In [15]:
import pandas as pd
import json
import openai

def convert_to_pipe_format(path_to_csv):
    df = pd.read_csv(path_to_csv)
    string = '/*\n'
    col_list = df.columns.values.tolist()
    string += 'col : ' + ' | '.join(df.columns) + '\n'
    for row_id, row in df.iterrows():
        string += f'row {row_id} : '
        for column_id, header in enumerate(df.columns):
            string += str(row[header])
            if column_id != len(df.columns) - 1:
                string += ' | '
        string += '\n'
    string += '*/\n'
    string += f'columns:{col_list}\n'
    return string

def generate_short_answer(table, question):
    answer_prompt = f"""
    Here is the table to answer this question. Answer the question in 3-4 words max.
    {table}
    Question: {question}
    The answer is: 
    """
    messages = [
        {"role": "system", "content": "You are an expert in answering questions from tabular data."},
        {"role": "user", "content": answer_prompt}
    ]
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0,
        messages=messages
    )
    generated_answer = completion.choices[0].message.content.strip()
    return generated_answer

def evaluate_qa_pair(qa_pair, correct_answers_list):
    try:
        table_name = qa_pair['table_name'].strip()  # Ensure no spaces in the name
        table_path = rf"D:\LLMTables\split\train\test_1\1\{table_name}.csv"
        
        # Attempt to convert the table
        table = convert_to_pipe_format(table_path)

    except Exception as e:
        print(f"Error processing table '{qa_pair['table_name']}': {e}")
        # Skip this QA pair
        return False

    question = qa_pair['question']
    generated_answer = generate_short_answer(table, question)
    correct_answer = qa_pair["result"]

    if generated_answer == correct_answer:
        correct_answers_list.append(qa_pair)
        print("----------------------------------------")
        print(question)
        print("actual: " + str(correct_answer))
        print("generated: " + generated_answer)
        print("----------------------------------------")
        return True
    else:
        print("----------------------------------------")
        print("incorrect answer")
        print(question)
        print("actual: " + str(correct_answer))
        print("generated: " + generated_answer)
        print("----------------------------------------")
        return False

    
def process_evaluation(json_data):
    total_questions = len(json_data)
    print("total questions: " + str(total_questions))
    correct_answers = 0
    incorrect_answers = []
    index = 1

    for qa_pair in json_data:
        print("question: " + str(index))
        if evaluate_qa_pair(qa_pair, correct_answers_list=[]):
            correct_answers += 1
        else:
            incorrect_answers.append(qa_pair)
        index += 1
    accuracy = (correct_answers / total_questions) * 100
    return correct_answers, accuracy, incorrect_answers

def save_incorrect_answers(incorrect_answers, output_path):
    with open(output_path, "w") as json_file:
        json.dump(incorrect_answers, json_file, indent=4)
    print(f"Incorrectly answered questions saved to {output_path}")

def evaluation_pipeline(input_json_path, incorrect_output_json_path):
    with open(input_json_path, "r") as file:
        json_data = json.load(file)
    correct_answers, accuracy, incorrect_answers = process_evaluation(json_data)
    print(f"Total Correct Answers: {correct_answers}")
    print(f"Accuracy: {accuracy:.2f}%")
    save_incorrect_answers(incorrect_answers, incorrect_output_json_path)

if __name__ == "__main__":
    input_json_path = r"D:\\LLMTables\\split\\train\\test_1\\1\\final_output.json"  
    incorrect_output_json_path = r"D:\LLMTables\Question Generation\test\incorrect_answers.json"  
    evaluation_pipeline(input_json_path, incorrect_output_json_path)

total questions: 60
question: 1
----------------------------------------
incorrect answer
Which cities have experienced either an attendance of more than 20,000 at sporting events or were part of the 2018 season?
actual: [('Boston',), ('Chicago',), ('Cleveland',), ('Detroit',), ('Milwaukee',), ('Minneapolis',), ('New York City',), ('Philadelphia',), ('Portland',), ('Toronto',), ('Washington',)]
generated: Philadelphia, Washington, Chicago, Toronto
----------------------------------------
question: 2
----------------------------------------
incorrect answer
In December 2017, which stadiums had a total attendance greater than 15,000 for sporting events?
actual: [(15300,), (16300,)]
generated: Capital One Arena, BMO Harris Bradley Center
----------------------------------------
question: 3
----------------------------------------
incorrect answer
What was the total attendance for sports events held in Washington since 2016, and was this total greater than 50,000?
actual: Error: a GROUP BY

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\LLMTables\\Question Generation\\test\\incorrect_answers.json'

In [1]:
like_eg = ["SELECT state, SUM(attendance) FROM sportset_2 WHERE stadium LIKE '%Center%' GROUP BY state HAVING SUM(attendance) > 30000",
            "SELECT COUNT(*) FROM [table] WHERE summary LIKE '%Boston Celtics%' AND month = 'February'",
            "SELECT stadium, MAX(capacity) FROM [table] WHERE stadium LIKE '%Arena%'",
            "SELECT * FROM [table] WHERE summary LIKE '%Los Angeles Lakers%'",
            "SELECT month, AVG(attendance) FROM sportset_coldtemp_30_13 WHERE summary LIKE '%Charlotte Hornets%' GROUP BY month HAVING AVG(attendance) > 15000",
            "SELECT city, COUNT(*) FROM sportset_midwest_30_8 WHERE summary LIKE '%Indiana Pacers%' AND month = 'February' GROUP BY city",
            "SELECT DISTINCT dayname FROM sportset_midwest_30_8 WHERE summary LIKE '%Rockets%' ORDER BY dayname ASC",
            "SELECT COUNT(*) FROM sportset_northeast_30_1 WHERE summary LIKE '%Los Angeles Lakers%' AND year = 2018",
            "SELECT dayname, COUNT(DISTINCT game_id) FROM sportset_2 WHERE summary LIKE '%Anthony Davis%' AND stadium LIKE '%Arena%' GROUP BY dayname ORDER BY COUNT(DISTINCT game_id) DESC",
            "SELECT stadium, city FROM [table] WHERE summary LIKE '%John Wall%' AND summary LIKE '%injury%",
            "SELECT COUNT(*) WHERE season = 2016 AND summary LIKE '%career - high%",
            "SELECT COUNT(DISTINCT game_id) FROM sportset_2 WHERE summary LIKE '%LeBron James%' AND attendance > capacity",
            ]

In [7]:
import random
test = random.sample(like_eg, 4)

In [8]:
test[0]

"SELECT COUNT(*) FROM [table] WHERE summary LIKE '%Boston Celtics%' AND month = 'February'"

In [11]:
import os
import shutil

def organize_csv_files(folder_path):
    # Get a list of all files in the folder
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # Sort the files to ensure consistent ordering
    all_files.sort()

    # Create subfolders and move files
    for idx, file in enumerate(all_files):
        # Determine the folder number (1-based)
        folder_number = idx // 10 + 1

        # Create the subfolder if it doesn't exist
        subfolder_path = os.path.join(folder_path, str(folder_number))
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Move the file to the subfolder
        source_file = os.path.join(folder_path, file)
        destination_file = os.path.join(subfolder_path, file)
        shutil.move(source_file, destination_file)

    print(f"Organized {len(all_files)} files into folders.")

# Replace 'your_folder_path' with the path to your folder containing the CSV files
folder_path = r"D:\\LLMTables\\split\\train\\40_60"
organize_csv_files(folder_path)


Organized 512 files into folders.
